In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

In [14]:
links = pd.read_csv('/home/benito/Desktop/Введение и классификация рекомендательных систем/Архив/links.csv')
movies = pd.read_csv('/home/benito/Desktop/Введение и классификация рекомендательных систем/Архив/movies.csv')
ratings = pd.read_csv('/home/benito/Desktop/Введение и классификация рекомендательных систем/Архив/ratings.csv')
tags = pd.read_csv('/home/benito/Desktop/Введение и классификация рекомендательных систем/Архив/tags.csv')

In [15]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [16]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
movies['comedy']=movies['genres'].apply(lambda row: 1 if row.find('Comedy')>-1 else 0)

In [20]:
movies=movies[movies['comedy']==1]

In [21]:
movies.head()

,movieId,title,genres,comedy
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
2,3,Grumpier Old Men (1995),Comedy|Romance,1
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1
4,5,Father of the Bride Part II (1995),Comedy,1
6,7,Sabrina (1995),Comedy|Romance,1


In [22]:
del movies['comedy']

In [23]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [24]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


Для составления рейтинга будем использовать теги и оценки пользователей.

In [30]:
joined_ratings = movies.join(ratings.set_index('movieId'), on='movieId')

In [55]:
# соединим уже созданную таблицу с таблицей с проставленными тегами по фильмам
joined_with_tags = joined_ratings.join(tags.set_index('movieId'), on='movieId', lsuffix='_left', rsuffix='_right')

In [56]:
joined_with_tags.head()

,movieId,title,genres,userId_left,rating,timestamp_left,userId_right,tag,timestamp_right
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,964982703.0,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,964982703.0,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,964982703.0,567.0,fun,1.525286e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,847434962.0,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,847434962.0,474.0,pixar,1.137207e+09


In [88]:
joined_with_tags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119183 entries, 0 to 9741
Data columns (total 9 columns):
movieId            119183 non-null int64
title              119183 non-null object
genres             119183 non-null object
userId_left        119180 non-null float64
rating             119180 non-null float64
timestamp_left     119180 non-null float64
userId_right       96904 non-null float64
tag                96904 non-null object
timestamp_right    96904 non-null float64
dtypes: float64(5), int64(1), object(3)
memory usage: 9.1+ MB


Первая метрика: a*(b-c)/(d-c) 
 - средний оценок по фильму = a
 - кол-во оценок  по фильму = b
 - минимальное кол-во оценок фильма = c
 - максимальное кол-во оценок фильма = d
 - (b-c)/(d-c) - нормированное кол-во оценов фильмов

In [66]:
# достанем простые статистики по количеству рейтингов
min_num_ratings = np.min([title_num_ratings[f] for f in title_num_ratings.keys()])
max_num_ratings = np.max([title_num_ratings[f] for f in title_num_ratings.keys()])
mean_num_ratings = np.mean([title_num_ratings[f] for f in title_num_ratings.keys()])
median_num_ratings = np.median([title_num_ratings[f] for f in title_num_ratings.keys()])

In [58]:
# считаем средний рейтинг на каждый фильм
title_mean_rating = {} # создаем пустой словарь

for title, group in tqdm_notebook(joined_ratings.groupby('title')): # для каждого фильма
    title_mean_rating[title] = group.rating.mean() # считаем средний рейтинг

In [67]:
film_with_our_mark = {} # создаем пустой словарь

# посчитаем нашу метрику для каждого фильма из датасета
for f in title_num_ratings.keys(): # для каждого фильма считаем метрику
    film_with_our_mark[f] = title_mean_rating[f] * (title_num_ratings[f] - min_num_ratings) / (max_num_ratings - min_num_ratings)

In [73]:
film_with_our_mark_list = [(k,film_with_our_mark[k]) for k in film_with_our_mark.keys()]# словарь в лист

In [74]:
# выводим топ 20
list(sorted(film_with_our_mark_list, key=lambda x: x[1], reverse=True))[:20]

[('Forrest Gump (1994)', 4.164133738601824),
 ('Aladdin (1992)', 2.104291616686659),
 ('Back to the Future (1985)', 2.0928719155612607),
 ("Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)", 1.5177337398373985),
 ('Ace Ventura: Pet Detective (1994)', 1.4831086199060748),
 ('Breakfast Club, The (1985)', 1.2903086552989425),
 ('Big Lebowski, The (1998)', 1.2563276576161988),
 ('Clerks (1994)', 1.2108055816135084),
 ('Batman Forever (1995)', 1.2090973829446323),
 ('Being John Malkovich (1999)', 1.1815410199556542),
 ('Austin Powers: The Spy Who Shagged Me (1999)', 1.1701269905261036),
 ('As Good as It Gets (1997)', 1.07104293699187),
 ('Austin Powers: International Man of Mystery (1997)', 1.0669664634146343),
 ('American Pie (1999)', 1.0506748756807955),
 ("Bug's Life, A (1998)", 0.9755600477200425),
 ('Blues Brothers, The (1980)', 0.9639953542392565),
 ('Blazing Saddles (1974)', 0.731904012588513),
 ('Bruce Almighty (2003)', 0.7078753005839917),
 ("Bridget Jones's Diary (2001)", 0.70

Вторая метрика: a*(b-c)/(d-c) 
 - средний оценок по фильму = a
 - кол-во юзеров, проставивших тег фильму = b
 - минимальное кол-во юзеров, проставивших тег фильму = c
 - максимальное кол-во юзеров, проставивших тег фильму = d
 - (b-c)/(d-c) - нормированное кол-во юзеров, проставивших тег фильму

In [77]:
# достанем по каждому фильму количество рейтингов
title_num_tags = {} # пустой словарь

for title, group in tqdm_notebook(joined_with_tags.groupby('title')): # для каждого фильма
    title_num_tags[title] = group.userId_right.unique().shape[0] # считаем кол-во юзеров,поставивших тег фильму

In [78]:
# достанем простые статистики по количеству тегов
min_num_tags = np.min([title_num_tags[f] for f in title_num_tags.keys()])
max_num_tags = np.max([title_num_tags[f] for f in title_num_tags.keys()])
mean_num_tags = np.mean([title_num_tags[f] for f in title_num_tags.keys()])
median_num_tags = np.median([title_num_tags[f] for f in title_num_tags.keys()])

In [81]:
film_with_second_mark = {} # создаем пустой словарь

# посчитаем нашу метрику для каждого фильма из датасета
for f in title_num_ratings.keys(): # для каждого фильма считаем метрику
    film_with_second_mark[f] = title_mean_rating[f] * (title_num_tags[f] - min_num_tags) / (max_num_tags - min_num_tags)

In [84]:
film_with_secind_mark_list = [(k,film_with_second_mark[k]) for k in film_with_second_mark.keys()]# словарь в лист

In [85]:
# выводим топ 20
list(sorted(film_with_secind_mark_list, key=lambda x: x[1], reverse=True))[:20]

[('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
  4.268041237113402),
 ('Anchorman: The Legend of Ron Burgundy (2004)', 2.8289473684210527),
 ('Forrest Gump (1994)', 2.082066869300912),
 ('Toy Story (1995)', 1.9604651162790698),
 ('Step Brothers (2008)', 1.7767857142857142),
 ('Corpse Bride (2005)', 1.7670454545454546),
 ('Happy Gilmore (1996)', 1.7196969696969697),
 ('Monty Python and the Holy Grail (1975)', 1.0404411764705883),
 ('Life Is Beautiful (La Vita è bella) (1997)', 1.0369318181818181),
 ('Fargo (1996)', 1.0290055248618784),
 ('Kiss Kiss Bang Bang (2005)', 1.0178571428571428),
 ('Trainspotting (1996)', 1.0098039215686274),
 ('Lost in Translation (2003)', 1.008445945945946),
 ('Three Colors: White (Trzy kolory: Bialy) (1994)', 1.00625),
 ('This Is Spinal Tap (1984)', 1.003787878787879),
 ('Finding Nemo (2003)', 0.9902482269503546),
 ('Big Lebowski, The (1998)', 0.9811320754716981),
 ('Wolf of Wall Street, The (2013)', 0.9791666666666666),
 ('T

Третья метрика: кол-во оценок, равных 5. Метрика позволит отобрать самые популярные и высоко оцененные фильмы

In [92]:
joined_ratings_5=joined_ratings[joined_ratings['rating']==5.0]

In [95]:
joined_ratings_5.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,31.0,5.0,850466616.0
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,40.0,5.0,832058959.0
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,43.0,5.0,848993983.0
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,46.0,5.0,834787906.0
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,57.0,5.0,965796031.0


In [96]:
# достанем по каждому фильму количество рейтингов
title_num_5 = {} # пустой словарь

for title, group in tqdm_notebook(joined_ratings_5.groupby('title')): # для каждого фильма
    title_num_5[title] = group.userId.unique().shape[0] # считаем кол-во юзеров,поставивших тег фильму

In [97]:
title_num_5_list = [(k,title_num_5[k]) for k in title_num_5.keys()]# словарь в лист

In [98]:
# выводим топ 20
list(sorted(title_num_5_list, key=lambda x: x[1], reverse=True))[:20]

[('Pulp Fiction (1994)', 123),
 ('Forrest Gump (1994)', 116),
 ('Fargo (1996)', 57),
 ('Princess Bride, The (1987)', 55),
 ('Monty Python and the Holy Grail (1975)', 51),
 ('Back to the Future (1985)', 49),
 ('Toy Story (1995)', 47),
 ("Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)", 43),
 ('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
  36),
 ('Aladdin (1992)', 35),
 ('Life Is Beautiful (La Vita è bella) (1997)', 35),
 ('Big Lebowski, The (1998)', 32),
 ('Groundhog Day (1993)', 32),
 ('Shrek (2001)', 30),
 ('Willy Wonka & the Chocolate Factory (1971)', 27),
 ('Snatch (2000)', 25),
 ('Graduate, The (1967)', 24),
 ('Trainspotting (1996)', 24),
 ('True Lies (1994)', 24),
 ('Being John Malkovich (1999)', 23)]